In [6]:
DEV = True
NewAidFlag = False
class Config():
    if DEV:
        TRAIN_FILE = "./data/train_df_v1_10_21_digit.csv"
        DEV_FILE = "./data/train_df_v1_22_digit.csv"
    else:
        TRAIN_FILE = "./data/train_df_v1_10_22_digit.csv"
        TEST_FILE = "./data/test_df_v1_digit.csv"
    

    MODE = "smape" #logloss  smape  mae

    NUM_SPLITS = 5
    RANDOM_SEED = 2019
    
    if NewAidFlag:
        FEATURE_COLS = [
            'billing_type', 
            'ad_type',
            'ad_type_convert_rate', 
            'material_size_convert_rate',
            'commodity_id_convert_rate',
            'ad_account_id','ad_industry_id',
            'commodity_id',
            'commodity_type',
            'material_size',
            'new_bid',
            'total_aid',
            'ad_industry_id_convert_rate',  
            'ad_account_id_convert_rate',

            'len_connectionType',
#             'len_consuptionAbility',
            'len_device', 'len_education', 
#             'len_gender', 
            'len_status',
            'len_work',
            'len_age', 'len_area', 
        ] 
#         for logloss
#         FEATURE_COLS = ['ad_account_id',
#                         'ad_industry_id_convert_rate', 
#            'ad_type_convert_rate',
#            'billing_type',
#            'commodity_type',
#                         'material_size', 
#                         'material_size_convert_rate',
#            'new_bid', 'total_aid',
                        
#                        'ad_type',
#                         'commodity_id_convert_rate'] 

    else: 
#         FEATURE_COLS = [   #'last_target','ad_id','billing_type', 'weekday', 'ad_type','ad_type_convert_rate','aid_convert_rate_t'
#             'billing_type',
#             'weekday',
# #             'ad_type',
#             'ad_type_convert_rate', 
# #             'material_size_convert_rate',
#             'commodity_id_convert_rate',
#             'ad_account_id','ad_industry_id',
#             'commodity_id',
# #            'commodity_type',
# #             'material_size',
#            'new_bid',
#             'total_aid',
#             'ad_industry_id_convert_rate',  
#             'ad_account_id_convert_rate',
#              'aid_convert_rate',
# #          'aid_convert_rate_max',
#        'aid_convert_rate_mean',
#         'aid_convert_rate_median',
# #        'aid_convert_rate_min',
# #             'target_max', 
#             'target_mean',
#             'target_median',
# #             'target_min'
#         ] 
        FEATURE_COLS = [   #'last_target','ad_id','billing_type', 'weekday', 'ad_type','ad_type_convert_rate','aid_convert_rate_t'
            'billing_type',
            'weekday',
            'ad_type',
            'ad_type_convert_rate', 
            'material_size_convert_rate',
            'commodity_id_convert_rate',
            'ad_account_id','ad_industry_id',
            'commodity_id',
           'commodity_type',
            'material_size',
           'new_bid',
            'total_aid',
            'ad_industry_id_convert_rate',  
            'ad_account_id_convert_rate',
             'aid_convert_rate',
#          'aid_convert_rate_max',
       'aid_convert_rate_mean',
        'aid_convert_rate_median',
#        'aid_convert_rate_min',
#             'target_max', 
            'target_mean',
            'target_median',
#             'target_min'
#              'len_connectionType',
# #             'len_consuptionAbility',
#             'len_device', 'len_education', 
# #             'len_gender', 
#             'len_status',
#             'len_work',
            'len_age', 'len_area',            
        ] 
        
# #         #for logloss   'aid_convert_rate', 'aid_convert_rate_t','ad_type','ad_account_id_convert_rate', 'commodity_id_convert_rate','commodity_id', 'ad_industry_id',
#         FEATURE_COLS = ['ad_account_id',
#                         'ad_industry_id_convert_rate', 
#            'ad_type_convert_rate',
#            'billing_type',
#            'commodity_type',
#                         'material_size', 
#                         'material_size_convert_rate',
#            'new_bid', 'total_aid',
                        
# #             'len_connectionType',
# # #             'len_consuptionAbility',
# #        'len_device', 'len_education', 
# # #             'len_gender', 
# #             'len_status', 'len_work',
# #              'len_age', 'len_area', 
#                        ] #,'weekday'
        

In [7]:
import pandas as pd

class FeatureDictionary(object):
    def __init__(self,trainfile=None,testfile=None,
                 dfTrain=None,dfTest=None,feature_cols=[]):
        assert not ((trainfile is None) and (dfTrain is None)), "trainfile or dfTrain at least one is set"
        assert not ((trainfile is not None) and (dfTrain is not None)), "only one can be set"
        assert not ((testfile is None) and (dfTest is None)), "testfile or dfTest at least one is set"
        assert not ((testfile is not None) and (dfTest is not None)), "only one can be set"

        self.trainfile = trainfile
        self.testfile = testfile
        self.dfTrain = dfTrain
        self.dfTest = dfTest

        self.feature_cols = feature_cols
        self.gen_feat_dict()

    def gen_feat_dict(self):
        if self.dfTrain is None:
            dfTrain = pd.read_csv(self.trainfile)
        else:
            dfTrain = self.dfTrain
        if self.dfTest is None:
            dfTest = pd.read_csv(self.testfile)
        else:
            dfTest = self.dfTest
        df = pd.concat([dfTrain,dfTest])
        self.feat_dict = {}
        tc = 0
        for col in df.columns:
            us = df[col].unique()
            self.feat_dict[col] = dict(zip(us,range(tc,len(us)+tc)))
            tc += len(us)
        self.feat_dim = tc

class DataParser(object):
    def __init__(self,feat_dict):
        self.feat_dict = feat_dict

    def parse(self,infile=None,df=None,has_label=False):
        assert not ((infile is None) and (df is None)), "infile or df at least one is set"
        assert not ((infile is not None) and (df is not None)), "only one can be set"

        if infile is None:
            dfi = df.copy()
        else:
            dfi = pd.read_csv(infile)
        
        if has_label:
            y = dfi['target'].values.tolist()
            dfi = pd.DataFrame(dfi,columns=self.feat_dict.feature_cols)
        else:
            ids = dfi['ad_id'].values.tolist()
            dfi = pd.DataFrame(dfi,columns=self.feat_dict.feature_cols)

        dfv = dfi.copy()
        for col in dfi.columns:
            dfi[col] = dfi[col].map(self.feat_dict.feat_dict[col])
            dfv[col] = 1.

        xi = dfi.values.tolist()
        xv = dfv.values.tolist()

        if has_label:
            return xi,xv,y
        else:
            return xi,xv,ids

In [8]:
import numpy as np
import tensorflow as tf

from time import time
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

class AFM(BaseEstimator, TransformerMixin):

    def __init__(self, feature_size, field_size,
                 embedding_size=8,
                 deep_layers=[32, 32], deep_init_size = 50,
                 dropout_deep=[0.5, 0.5, 0.5],
                 deep_layer_activation=tf.nn.relu,
                 epoch=10, batch_size=256,
                 learning_rate=0.001, optimizer="adam",
                 batch_norm=0, batch_norm_decay=0.995,
                 verbose=False, random_seed=2016,
                 loss_type="logloss", eval_metric=roc_auc_score,
                greater_is_better=True,
                 use_inner=True):
        assert loss_type in ["logloss", "mse", "smape", "mae"], \
            "loss_type can be either 'logloss' for classification task or 'mse' for regression task"

        self.feature_size = feature_size
        self.field_size = field_size
        self.embedding_size = embedding_size

        self.deep_layers = deep_layers
        self.deep_init_size = deep_init_size
        self.dropout_dep = dropout_deep
        self.deep_layers_activation = deep_layer_activation

        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.optimizer_type = optimizer

        self.batch_norm = batch_norm
        self.batch_norm_decay = batch_norm_decay

        self.verbose = verbose
        self.random_seed = random_seed
        self.loss_type = loss_type
        self.eval_metric = eval_metric
        self.greater_is_better = greater_is_better
        self.train_result,self.valid_result = [],[]

        self.use_inner = use_inner

        self._init_graph()

    def _init_graph(self):
        self.graph = tf.Graph()
        with self.graph.as_default():
            tf.set_random_seed(self.random_seed)

            self.feat_index = tf.placeholder(tf.int32,
                                             shape=[None,None],
                                             name='feat_index')
            self.feat_value = tf.placeholder(tf.float32,
                                           shape=[None,None],
                                           name='feat_value')

            self.label = tf.placeholder(tf.float32,shape=[None,1],name='label')
            self.dropout_keep_deep = tf.placeholder(tf.float32,shape=[None],name='dropout_deep_deep')
            self.train_phase = tf.placeholder(tf.bool,name='train_phase')

            self.weights = self._initialize_weights()

            # Embeddings
            self.embeddings = tf.nn.embedding_lookup(self.weights['feature_embeddings'],self.feat_index) # N * F * K
            feat_value = tf.reshape(self.feat_value,shape=[-1,self.field_size,1])
            self.embeddings = tf.multiply(self.embeddings,feat_value) # N * F * K

            # first order term
            self.y_first_order = tf.nn.embedding_lookup(self.weights['feature_bias'], self.feat_index)
            self.y_first_order = tf.reduce_sum(tf.multiply(self.y_first_order, feat_value), 2)

            # second order term
            # sum-square-part
            self.summed_features_emb = tf.reduce_sum(self.embeddings, 1)  # None * k
            self.summed_features_emb_square = tf.square(self.summed_features_emb)  # None * K

            # squre-sum-part
            self.squared_features_emb = tf.square(self.embeddings)
            self.squared_sum_features_emb = tf.reduce_sum(self.squared_features_emb, 1)  # None * K

            # second order
            self.y_second_order = 0.5 * tf.subtract(self.summed_features_emb_square, self.squared_sum_features_emb)
            
            # Deep component   self.y_deep = tf.layers.batch_normalization(self.y_deep, training=self.train_phase)
            self.y_deep = self.y_second_order
            #self.y_deep = tf.layers.batch_normalization(self.y_deep, training=self.train_phase)
            for i in range(0, len(self.deep_layers)):
                self.y_deep = tf.add(tf.matmul(self.y_deep, self.weights["layer_%d" % i]), self.weights["bias_%d" % i])
                self.y_deep = self.deep_layers_activation(self.y_deep)
                self.y_deep = tf.nn.dropout(self.y_deep, self.dropout_keep_deep[i + 1])
                #self.y_deep = tf.layers.batch_normalization(self.y_deep, training=self.train_phase)

            # bias
            self.y_bias = self.weights['bias'] * tf.ones_like(self.label)
            # out
            self.out = tf.add_n([tf.reduce_sum(self.y_first_order,axis=1,keep_dims=True),
                                 tf.reduce_sum(self.y_deep,axis=1,keep_dims=True),
                                 self.y_bias])

            # loss
            if self.loss_type == "logloss":
                self.out = tf.nn.sigmoid(self.out)
                self.loss = tf.losses.log_loss(self.label, self.out)
            elif self.loss_type == "mse":
                self.loss = tf.nn.l2_loss(tf.subtract(self.label, self.out))
            elif self.loss_type == "smape":
                self.loss = 2.0 * tf.reduce_sum(tf.abs(self.label - self.out) / (tf.abs(self.label) + tf.abs(self.out) + 0.00001)) * 100
            elif self.loss_type == "mae":
                self.loss = tf.reduce_sum(tf.abs(self.label - self.out)) 
                
            if self.optimizer_type == "adam":
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate, beta1=0.9, beta2=0.999,
                                                        epsilon=1e-8).minimize(self.loss)
            elif self.optimizer_type == "adagrad":
                self.optimizer = tf.train.AdagradOptimizer(learning_rate=self.learning_rate,
                                                           initial_accumulator_value=1e-8).minimize(self.loss)
            elif self.optimizer_type == "gd":
                self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
            elif self.optimizer_type == "momentum":
                self.optimizer = tf.train.MomentumOptimizer(learning_rate=self.learning_rate, momentum=0.95).minimize(
                    self.loss)

            #init
            self.saver = tf.train.Saver()
            init = tf.global_variables_initializer()
            self.sess = tf.Session()
            self.sess.run(init)

            # number of params
            total_parameters = 0
            for variable in self.weights.values():
                shape = variable.get_shape()
                variable_parameters = 1
                for dim in shape:
                    variable_parameters *= dim.value
                total_parameters += variable_parameters
            if self.verbose > 0:
                print("#params: %d" % total_parameters)

    def _initialize_weights(self):
        weights = dict()

        #embeddings
        weights['feature_embeddings'] = tf.Variable(
            tf.random_normal([self.feature_size,self.embedding_size],0.0,0.01),
            name='feature_embeddings')
        weights['feature_bias'] = tf.Variable(tf.random_normal([self.feature_size,1],0.0,1.0),name='feature_bias')
        weights['bias'] = tf.Variable(tf.constant(0.1),name='bias')

        #deep layers
        num_layer = len(self.deep_layers)
        input_size = self.embedding_size
        glorot = np.sqrt(2.0/(input_size + self.deep_layers[0]))

        weights['layer_0'] = tf.Variable(
            np.random.normal(loc=0,scale=glorot,size=(input_size,self.deep_layers[0])),dtype=np.float32)
        weights['bias_0'] = tf.Variable(
            np.random.normal(loc=0,scale=glorot,size=(1,self.deep_layers[0])),dtype=np.float32)

        for i in range(1,num_layer):
            glorot = np.sqrt(2.0 / (self.deep_layers[i - 1] + self.deep_layers[i]))
            weights["layer_%d" % i] = tf.Variable(
                np.random.normal(loc=0, scale=glorot, size=(self.deep_layers[i - 1], self.deep_layers[i])),
                dtype=np.float32)  # layers[i-1] * layers[i]
            weights["bias_%d" % i] = tf.Variable(
                np.random.normal(loc=0, scale=glorot, size=(1, self.deep_layers[i])),
                dtype=np.float32)  # 1 * layer[i]

        return weights

    def get_batch(self,Xi,Xv,y,batch_size,index):
        start = index * batch_size
        end = (index + 1) * batch_size
        end = end if end < len(y) else len(y)
        return Xi[start:end],Xv[start:end],[[y_] for y_ in y[start:end]]

    # shuffle three lists simutaneously
    def shuffle_in_unison_scary(self, a, b, c):
        rng_state = np.random.get_state()
        np.random.shuffle(a)
        np.random.set_state(rng_state)
        np.random.shuffle(b)
        np.random.set_state(rng_state)
        np.random.shuffle(c)

    def smape(self,y_true, y_pred):
        return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
    
    def predict(self, Xi, Xv,y):
        """
        :param Xi: list of list of feature indices of each sample in the dataset
        :param Xv: list of list of feature values of each sample in the dataset
        :return: predicted probability of each sample
        """
        feed_dict = {self.feat_index: Xi,
                     self.feat_value: Xv,
                     self.label: y,
                     self.dropout_keep_deep: [1.0] * len(self.dropout_dep),
                     self.train_phase: False}

        out = self.sess.run(self.out, feed_dict=feed_dict)
        return out

    def fit_on_batch(self,Xi,Xv,y):
        feed_dict = {self.feat_index:Xi,
                     self.feat_value:Xv,
                     self.label:y,
                     self.dropout_keep_deep:self.dropout_dep,
                     self.train_phase:True}

        loss,opt = self.sess.run([self.loss,self.optimizer],feed_dict=feed_dict)

        return loss

    def fit(self, Xi_train, Xv_train, y_train,
            Xi_valid=None, Xv_valid=None, y_valid=None,
            early_stopping=False, refit=False):

        has_valid = Xv_valid is not None
        best_loss = 200
        for epoch in range(self.epoch):
            t1 = time()
            self.shuffle_in_unison_scary(Xi_train, Xv_train, y_train)
            total_batch = int(len(y_train) / self.batch_size)
            for i in range(total_batch):
                Xi_batch, Xv_batch, y_batch = self.get_batch(Xi_train, Xv_train, y_train, self.batch_size, i)
                self.fit_on_batch(Xi_batch, Xv_batch, y_batch)
            if has_valid:
                y_valid = np.array(y_valid).reshape((-1,1))
                out = self.predict(Xi_valid, Xv_valid, y_valid)
                if self.loss_type=='logloss':
                    print("epoch: {0}, train logloss: {1:.6f}".format(epoch + 1, roc_auc_score(y_valid,out)))
                else:
                    loss = self.smape(y_valid, out)
                    best_loss = min(best_loss,loss)
                    print("epoch: {0}, train loss: {1:.6f}".format(epoch + 1, loss))
        return best_loss

In [9]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from matplotlib import pyplot as plt

config = Config()
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
def get_dev(dev,old_aid_set,NewAidFlag):
    old_index = []
    new_index = []
    _dev = dev.copy()
    for index, row in tqdm(_dev.iterrows()):
        if row['ad_id'] in old_aid_set:
            old_index.append(index)
        else:
            new_index.append(index)
    if NewAidFlag:
        return _dev.loc[new_index]
    else:
        return _dev.loc[old_index]
    
def load_data():
    dfTrain = pd.read_csv(config.TRAIN_FILE)
    print(dfTrain.shape)
    if DEV:
        dfDev = pd.read_csv(config.DEV_FILE)
        print(dfDev.shape)
        if config.MODE == 'logloss':
            old_aid_set = set(dfTrain.ad_id)
            if NewAidFlag:
                dfDev = get_dev(dfDev,old_aid_set,NewAidFlag)
                dfDev.reset_index(drop=True,inplace=True)
            else:
                dfDev = get_dev(dfDev,old_aid_set,NewAidFlag)
                dfDev.reset_index(drop=True,inplace=True)
            dfDev=dfDev[dfDev['target'] >= 1]
            dfDev.reset_index(drop=True,inplace=True)
            dfDev['target'] = dfDev.target.apply(lambda x : 1 if x <= 1 else 0)
        else:
            old_aid_set = set(dfTrain.ad_id)
            if NewAidFlag:
                dfDev = get_dev(dfDev,old_aid_set,NewAidFlag)
                dfDev.reset_index(drop=True,inplace=True)
            else:
                dfDev = get_dev(dfDev,old_aid_set,NewAidFlag)
                dfDev.reset_index(drop=True,inplace=True)
            dfDev=dfDev[dfDev['target'] >= 1]
            dfDev.reset_index(drop=True,inplace=True)
        print(dfDev.shape)
    else:
        dfTest = pd.read_csv(config.TEST_FILE)
    if config.MODE == 'logloss':
        dfTrain['target'] = dfTrain.target.apply(lambda x : 1 if x <= 1 else 0)
    else:
        dfTrain['target'] = dfTrain.target.apply(lambda x : 1 if x <= 1 else x)
#         pass

    cols = config.FEATURE_COLS

    X_train = dfTrain[cols].values
    y_train = dfTrain['target'].values
    
    if DEV:
        X_dev = dfDev[cols].values
        y_dev = dfDev['target'].values
    else:
        X_test = dfTest[cols].values
        
    if DEV:
        return dfTrain,dfDev,X_train,y_train,X_dev,y_dev
    else:
        return dfTrain,dfTest,X_train,y_train,X_test

def run_dev_model_nfm(dfTrain,dfDev,pnn_params):
    print('run_dev_model_nfm')
    fd = FeatureDictionary(dfTrain=dfTrain,
                           dfTest=dfDev,
                           feature_cols=config.FEATURE_COLS)
    data_parser = DataParser(feat_dict= fd)
    Xi_train,Xv_train,y_train = data_parser.parse(df=dfTrain,has_label=True)
    Xi_dev,Xv_dev,y_dev = data_parser.parse(df=dfDev,has_label=True)
    pnn_params['feature_size'] = fd.feat_dim
    pnn_params['field_size'] = len(Xi_train[0])
    nfm = AFM(**pnn_params)
    loss = nfm.fit(Xi_train, Xv_train, y_train, Xi_dev,Xv_dev,y_dev)
    y_dev = np.array(y_dev).reshape((-1,1))
    out = nfm.predict(Xi_dev, Xv_dev, y_dev)    
    return out

def run_test_model_nfm(dfTrain,dfTest,pnn_params):
    print('run_test_model_nfm')
    fd = FeatureDictionary(dfTrain=dfTrain,
                           dfTest=dfTest,
                           feature_cols=config.FEATURE_COLS)
    data_parser = DataParser(feat_dict= fd)
    Xi_train,Xv_train,y_train = data_parser.parse(df=dfTrain,has_label=True)
    Xi_test,Xv_test,_ = data_parser.parse(df=dfTest)
    pnn_params['feature_size'] = fd.feat_dim
    pnn_params['field_size'] = len(Xi_train[0])
    nfm = AFM(**pnn_params)
    loss = nfm.fit(Xi_train, Xv_train, y_train, Xi_train[:10000],Xv_train[:10000],y_train[:10000])
    y_test = np.zeros(len(Xi_test))
    y_test = np.array(y_test).reshape((-1,1))
    y_test = nfm.predict(Xi_test, Xv_test, y_test)
    return y_test

In [10]:
if config.MODE == 'logloss':
    pnn_params = {
        "embedding_size":16,
        "deep_layers":[256,128],
        "dropout_deep":[0.6,0.6,0.6], # old ad_id 0.8
        "deep_layer_activation":tf.nn.relu,
        "epoch":12,#old ad 35    new ad 5
        "batch_size":1024,
        "learning_rate":0.0005,   #old ad 0.001
        "optimizer":"adam",
        "loss_type":config.MODE, #"logloss","smape"
        "batch_norm":1,
        "batch_norm_decay":0.995,
        "verbose":True,
        "random_seed":config.RANDOM_SEED,
        "deep_init_size":50,
        "use_inner":False
    }
else:
    if NewAidFlag:
        #new aid
        pnn_params = {
            "embedding_size":8,
            "deep_layers":[128,64],
            "dropout_deep":[0.8,0.8,0.8], # old ad_id 0.8
            "deep_layer_activation":tf.nn.relu,
            "epoch":10,#old ad 35    new ad 5
            "batch_size":1024,
            "learning_rate":0.001,   #old ad 0.001
            "optimizer":"adam",
            "loss_type":config.MODE, #"logloss","smape"
            "batch_norm":1,
            "batch_norm_decay":0.995,
            "verbose":True,
            "random_seed":config.RANDOM_SEED,
            "deep_init_size":50,
            "use_inner":False
        }
    else:
        #lod aid
        pnn_params = {
            "embedding_size":16,
            "deep_layers":[256,128],
            "dropout_deep":[0.7,0.7,0.7], # old ad_id 0.8
            "deep_layer_activation":tf.nn.relu,
            "epoch":15,#old ad 35    new ad 5
            "batch_size":1024,
            "learning_rate":0.003,   #old ad 0.001
            "optimizer":"adam",
            "loss_type":config.MODE, #"logloss","smape"
            "batch_norm":1,
            "batch_norm_decay":0.995,
            "verbose":True,
            "random_seed":config.RANDOM_SEED,
            "deep_init_size":50,
            "use_inner":False
        }
if DEV:
    dfTrain,dfDev,X_train, y_train,X_dev,y_dev = load_data()
    predict_dev = run_dev_model_nfm(dfTrain,dfDev,pnn_params)
else:
    dfTrain,dfTest,X_train,y_train,X_test = load_data()
    predict_test = run_test_model_nfm(dfTrain,dfTest,pnn_params)

(677738, 37)
(57380, 37)


57380it [00:02, 21467.72it/s]


(14508, 37)
run_dev_model_nfm
#params: 3813595
epoch: 1, train loss: 65.393220
epoch: 2, train loss: 63.044441
epoch: 3, train loss: 61.821918
epoch: 4, train loss: 62.207769
epoch: 5, train loss: 61.336822
epoch: 6, train loss: 61.137027
epoch: 7, train loss: 60.655239
epoch: 8, train loss: 60.687061
epoch: 9, train loss: 60.377623
epoch: 10, train loss: 60.890567
epoch: 11, train loss: 60.113732
epoch: 12, train loss: 60.299488
epoch: 13, train loss: 59.907301
epoch: 14, train loss: 60.018637
epoch: 15, train loss: 60.185020


In [11]:
test_dev = pd.DataFrame(dfDev,columns=['ad_id','target'])
test_dev['pred'] = predict_dev
test_dev['pred'] = test_dev['pred'].apply(lambda x : 1 if x < 1 else x)
print('smape test_dev',smape(test_dev['target'],test_dev['pred']))

smape test_dev 59.09294105321281


In [12]:
test_dev.describe()

,ad_id,target,pred
count,14508.000000,14508.000000,14508.000000
mean,129693.615247,19.490419,19.230201
std,74940.405202,115.204132,89.493490
min,6.000000,1.000000,1.000000
25%,64690.000000,1.000000,1.003653
50%,129944.500000,3.000000,2.903118
75%,194589.500000,8.000000,9.497584
max,259530.000000,7859.000000,2713.360596
